# analyzing ecommerce transactions using spark SQL

**SCENARIO**
Imagine you are working as a Data Engineer for an e-commerce company like Amazon. The
company wants to analyze customer transactions and product details to:
• Calculate total revenue per category
• Identify high-value customers
• Find the most popular product categories
• Compare new and existing customers
We will use Spark SQL to process and analyze this data efficiently.

# TASK 1: Create  tables
- customers table stores details like customer_id,name,country,registration_Date
- helps analyze new vs existing customers


- transactions table stores purchase details: transaction_id, customer_id, product_id, amount, 
date- • Helps analyze revenue, customer spending, and tren


- products table stores product details: product_id, category, price-• Helps analyze popular products and revenue per categoryds

In [1]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName("Ecommerce_Analysis").getOrCreate()

#  Create Customers Table
customers_data = [
 (1, "Alice", "USA", "2022-01-10"),
 (2, "Bob", "Canada", "2021-05-15"),
 (3, "Charlie", "UK", "2023-07-20"),
 (4, "David", "Germany", "2022-09-30"),
 (5, "Eve", "France", "2021-11-25"),
]
customers_columns = ["customer_id", "name", "country", "registration_date"]
customers_df = spark.createDataFrame(customers_data, customers_columns)
customers_df.createOrReplaceTempView("customers")

#  Create Transactions Table
transactions_data = [
 (101, 1, 201, 100.0, "2024-01-10"),
 (102, 2, 202, 250.0, "2024-01-12"),
 (103, 3, 203, 300.0, "2024-01-15"),
 (104, 4, 201, 150.0, "2024-01-18"),
 (105, 5, 204, 500.0, "2024-01-20"),
 (106, 1, 202, 200.0, "2024-01-22"),
]
transactions_columns = ["transaction_id", "customer_id", "product_id",
"amount", "date"]
transactions_df = spark.createDataFrame(transactions_data,
transactions_columns)
transactions_df.createOrReplaceTempView("transactions")

# Create Products Table
products_data = [
 (201, "Electronics", 100),
 (202, "Clothing", 200),
 (203, "Books", 300),
 (204, "Home Appliances", 500),
]
products_columns = ["product_id", "category", "price"]
products_df = spark.createDataFrame(products_data, products_columns)
products_df.createOrReplaceTempView("products")


# TASK 2: SPARK SQL for ecommerce analysis

In [2]:
# Total Revenue per Product Category
revenue_query = """
 SELECT p.category, SUM(t.amount) AS total_revenue
 FROM transactions t
 JOIN products p ON t.product_id = p.product_id
 GROUP BY p.category
 ORDER BY total_revenue DESC
"""
revenue_df = spark.sql(revenue_query)
print("Total Revenue per Category:")
revenue_df.show()

Total Revenue per Category:
+---------------+-------------+
|       category|total_revenue|
+---------------+-------------+
|Home Appliances|        500.0|
|       Clothing|        450.0|
|          Books|        300.0|
|    Electronics|        250.0|
+---------------+-------------+



In [3]:
# Identify High-Value Customers (Customers Who Spent More Than $300)
high_value_customers_query = """
 SELECT c.customer_id, c.name, c.country, SUM(t.amount) AS total_spent
 FROM transactions t
 JOIN customers c ON t.customer_id = c.customer_id
 GROUP BY c.customer_id, c.name, c.country
 HAVING total_spent > 300
 ORDER BY total_spent DESC
"""
high_value_customers_df = spark.sql(high_value_customers_query)
print("High-Value Customers:")
high_value_customers_df.show()

High-Value Customers:
+-----------+----+-------+-----------+
|customer_id|name|country|total_spent|
+-----------+----+-------+-----------+
|          5| Eve| France|      500.0|
+-----------+----+-------+-----------+



In [5]:
#  Find the Most Popular Product Category (Based on Number of Purchases)
popular_category_query = """
 SELECT p.category, COUNT(t.transaction_id) AS num_purchases
 FROM transactions t
 JOIN products p ON t.product_id = p.product_id
 GROUP BY p.category
 ORDER BY num_purchases DESC
"""
popular_category_df = spark.sql(popular_category_query)
print("Most Popular Product Categories:")
popular_category_df.show()


Most Popular Product Categories:
+---------------+-------------+
|       category|num_purchases|
+---------------+-------------+
|    Electronics|            2|
|       Clothing|            2|
|          Books|            1|
|Home Appliances|            1|
+---------------+-------------+



In [6]:
# Compare New vs. Existing Customers
new_vs_existing_query = """
 SELECT
 CASE
 WHEN c.registration_date >= '2023-01-01' THEN 'New Customer'
 ELSE 'Existing Customer'
 END AS customer_type,
 COUNT(DISTINCT t.customer_id) AS customer_count,
 SUM(t.amount) AS total_spent
 FROM transactions t
 JOIN customers c ON t.customer_id = c.customer_id
 GROUP BY customer_type
"""
new_vs_existing_df = spark.sql(new_vs_existing_query)
print("Comparison of New vs. Existing Customers:")
new_vs_existing_df.show()

Comparison of New vs. Existing Customers:
+-----------------+--------------+-----------+
|    customer_type|customer_count|total_spent|
+-----------------+--------------+-----------+
|Existing Customer|             4|     1200.0|
|     New Customer|             1|      300.0|
+-----------------+--------------+-----------+



# Summary of insights
- Home Appliances generate the most revenue but are not the most frequently
purchased.- 
• High-value customers like Eve (France) should be targeted with premium promotions-
• Clothing & Electronics are the most purchased product categorig- 5
• Existing customers contribute more revenue, but new customers need incentives to
increase spending.